# Analyze FMCIB features for original and READII negative control CT images

This notebook is set up to use outputs from the `run_fmcib.ipynb` notebook.

Image features extracted from CT images cropped to a Gross Tumour Volume (GTV) undergo correlation analysis. Results are compared across READII negative control image types.

## Set up pixi environment kernel

1. Run the following commands in the terminal:

    ```bash
    $ pixi install

    $ pixi run make_kernel
    ```

2. In the `Select Kernel` menu at the top right of the notebook, select `Jupyter Kernel` as the source. 

3. Refresh the options and one called `readii-fmcib` should appear. Select this option.

## Imports

In [1]:
import itertools
import pandas as pd

from pathlib import Path

from readii.utils import logger
from readii.io.loaders import loadImageDatasetConfig, loadFeatureFilesFromImageTypes
from readii.io.writers.correlation_writer import CorrelationWriter
from readii.analyze.correlation import getFeatureCorrelations 


import sys; sys.path.append("code")
from analyze import prepPatientIndex, makeAllHeatmapPlots, makeAllHistogramPlots

## Initialize dataset name and load config 

In [2]:
config = loadImageDatasetConfig("RADCURE", Path("config"))

DATASET_NAME = config["dataset_name"]
PAT_ID_PATTERN = config['patient_id_pattern']
NEG_CONTROL_REGIONS = config["negative_control_regions"]
NEG_CONTROL_TYPES = config["negative_control_types"]

# Get full list of image types to run FMCIB on
negative_control_list = [f"{negative_control[0]}_{negative_control[1]}" for negative_control in itertools.product(NEG_CONTROL_TYPES, NEG_CONTROL_REGIONS)]

## Set up data directories

In [3]:
correlations_dir = Path("results", DATASET_NAME, "analysis", "correlations")

# Set features input directory
features_dir = Path("results", DATASET_NAME, "fmcib_features")

# Make correlation results output directory
for combo in itertools.product([correlations_dir], ["matrix", "heatmap", "histogram"]):
    Path(*combo).mkdir(parents=True, exist_ok=True)

## Load all extracted feature sets

In [4]:
# Load the extracted feature data
# This makes a dictionary of feature sets, with the image type as the key
extracted_feature_sets = loadFeatureFilesFromImageTypes(extracted_feature_dir=features_dir,
                                                        image_types = (["original"] + negative_control_list), 
                                                        drop_labels = False)

# Run correlation analysis for each image type

In [5]:
# Set up writers for correlation and plots
corr_matrix_writer = CorrelationWriter(
    root_directory = correlations_dir / "matrix",
    filename_format = DATASET_NAME + "_{VerticalFeatureType}_{HorizontalFeatureType}_{CorrelationType}_correlations.csv",
    overwrite = False,
    create_dirs = True
)

In [ ]:
# Flag to make the original self-correlation plots only once
make_original_plots = True

# Name of the column used to extract the patient ID for a row of features
file_path_column = 'image_path'

# Correlation method to apply
correlation_method = "pearson"

# Colormap to use for plots
heatmap_cmap = "nipy_spectral"

# Get and set up the feature dataframe for the original features once
vertical_feature_type = "original"
vertical_features_df = prepPatientIndex(extracted_feature_sets[vertical_feature_type],
                                        file_path_column,
                                        PAT_ID_PATTERN)

# Iterate over each negative control feature set and perform correlation analysis
for horizontal_feature_type in negative_control_list:
    print(f"Processing {vertical_feature_type} vs {horizontal_feature_type} correlations.")

    corr_matrix_output_path = corr_matrix_writer.resolve_path(VerticalFeatureType=vertical_feature_type,
                                                              HorizontalFeatureType=horizontal_feature_type,
                                                              CorrelationType=correlation_method)
    
    # Get extracted features for this image type, extract set the patient ID as the dataframe index, remove image_path column
    horizontal_features_df = prepPatientIndex(extracted_feature_sets[horizontal_feature_type], 
                                                file_path_column = file_path_column, 
                                                pat_id_pattern = PAT_ID_PATTERN)

    
    # Load existing correlation matrix if it's available
    if corr_matrix_output_path.exists() and corr_matrix_output_path.is_file():
        print("     Loading correlation matrix.")
        feature_correlation_df = pd.read_csv(corr_matrix_output_path, index_col=0)
    
    # Calculate the correlation matrix if the file doesn't exist
    else:
        print("     Calculating correlation matrix.")
        # Calculate correlations between original image features and image type features
        feature_correlation_df = getFeatureCorrelations(vertical_features=vertical_features_df,
                                                        horizontal_features=horizontal_features_df,
                                                        vertical_feature_name=vertical_feature_type,
                                                        horizontal_feature_name=horizontal_feature_type,
                                                        method = correlation_method)
        # save out the correlation dataframe
        corr_matrix_writer.save(feature_correlation_df, 
                                VerticalFeatureType=vertical_feature_type,
                                HorizontalFeatureType=horizontal_feature_type,
                                CorrelationType=correlation_method)
        
    print("     Generating heatmaps for correlations.")
    vert_heatmap_path, horiz_heatmap_path, cross_heatmap_path = makeAllHeatmapPlots(feature_correlation_df,
                                                                                    vertical_feature_type,
                                                                                    horizontal_feature_type,
                                                                                    correlations_dir,
                                                                                    correlation_method,
                                                                                    heatmap_cmap)
    
    print("     Generating histograms for correlations.")
    vert_histogram_path, horiz_histogram_path, cross_histogram_path = makeAllHistogramPlots(feature_correlation_df,
                                                                                            vertical_feature_type,
                                                                                            horizontal_feature_type,
                                                                                            correlations_dir,
                                                                                            correlation_method,
                                                                                            num_bins=450
                                                                                            )

Processing original vs shuffled_roi correlations.
     Loading correlation matrix.
     Generating heatmaps for correlations.
Plotting vertical feature correlations heatmap...


2024-12-30T17:27:33-0500 [warning  ] File results/RADCURE/analysis/correlations/heatmap/nipy_spectral/original_pearson_self_correlation_heatmap.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHeatmap:73


Plotting horizontal feature correlations heatmap...


2024-12-30T17:27:44-0500 [warning  ] File results/RADCURE/analysis/correlations/heatmap/nipy_spectral/shuffled_roi_pearson_self_correlation_heatmap.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHeatmap:73


Plotting cross feature correlations heatmap...


...

2024-12-30T17:27:53-0500 [warning  ] File results/RADCURE/analysis/correlations/histogram/original_pearson_self_correlation_histogram.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHistogram:132


Plotting horizontal feature correlations histogram...


2024-12-30T17:27:53-0500 [warning  ] File results/RADCURE/analysis/correlations/histogram/shuffled_roi_pearson_self_correlation_histogram.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHistogram:132


Plotting cross feature correlations histogram...


2024-12-30T17:27:54-0500 [warning  ] File results/RADCURE/analysis/correlations/histogram/original_vs_shuffled_roi_pearson_cross_correlation_histogram.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHistogram:132
